## **LOCACIONES PLANTAS CLIENTES: Restricciones Adicionales**

In [1]:
from docplex.mp.model import Model
import pandas

In [2]:
df_costo_cliente_planta = pandas.read_excel("./data.xlsx", sheet_name="COSTO CLIENTE-PLANTA")
df_clientes = pandas.read_excel("./data.xlsx", sheet_name="DEMANDAS CLIENTES")
df_plantas = pandas.read_excel("./data.xlsx", sheet_name="COSTO Y CAPACIDAD PLANTAS")

**Conjuntos**

In [3]:
PLANTAS = df_plantas["Planta"].to_list()
CLIENTES = df_clientes["Cliente"].to_list()

**Parámetros**

In [4]:
COSTO_CLIENTE_PLANTA = df_costo_cliente_planta.set_index(["Planta", "Cliente"])['Costo de asignar un cliente a un planta'].to_dict()
DEMANDA_CLIENTE = df_clientes.set_index(["Cliente"])["Demanda del cliente"].to_dict()
COSTO_ABRIR_PLANTA = df_plantas.set_index(["Planta"])["Costo de abrir planta"].to_dict()
CAPACIDAD_PLANTA = df_plantas.set_index(["Planta"])["Capacidad máxima"].to_dict()

In [5]:
model = Model()

**Variables**

In [6]:
x = model.binary_var_dict(PLANTAS, name="x")
y = model.binary_var_dict([(planta, cliente) for planta in PLANTAS for cliente in CLIENTES], name="y")

alpha = model.binary_var_dict(PLANTAS, name="alpha")

**Función Objetivo**

In [7]:
costo_abrir_plantas = model.sum(x[planta] * COSTO_ABRIR_PLANTA[planta] for planta in PLANTAS)
costo_cliente_planta = model.sum(y[(planta, cliente)] * COSTO_CLIENTE_PLANTA[(planta, cliente)] for planta in PLANTAS for cliente in CLIENTES)
costo_4plantas = model.sum(alpha[planta] * 2000 for planta in PLANTAS)
model.minimize(costo_abrir_plantas + costo_cliente_planta + costo_4plantas)

**Restricciones**

In [8]:
for planta in PLANTAS:
    for cliente in CLIENTES:
        model.add_constraint(y[(planta, cliente)] <= x[planta], ctname=f"ct1_{planta}_{cliente}")

for planta in PLANTAS:
    model.add_constraint(model.sum(y[(planta, cliente)] * DEMANDA_CLIENTE[cliente] for cliente in CLIENTES) <= CAPACIDAD_PLANTA[planta], ctname=f"ct2_{planta}")

for cliente in CLIENTES:
    model.add_constraint(model.sum(y[(planta, cliente)] for planta in PLANTAS) == 1, ctname=f"ct3_{cliente}")

for planta in PLANTAS:
    model.add_constraint(model.sum(y[(planta, cliente)] for cliente in CLIENTES) >= 3 * x[planta], ctname=f"ct4_{planta}")

for planta in PLANTAS:
    model.add_constraint(model.sum(y[(planta, cliente)] * DEMANDA_CLIENTE[cliente] for cliente in CLIENTES) >= 0.9 * CAPACIDAD_PLANTA[planta] * x[planta], ctname=f"ct5_{planta}")

$$ \forall p\in P:  Y_{p,7} + Y_{p,10}+ Y_{p,49} \le 1 $$

$$ \sum_{c\in C: c \le 10} Y_{12,c} \ge 2 $$

$$ \forall p\in P: \mathrm{Si } \sum_{c\in C} Y_{pc} \ge 4\to \alpha_{p}=1 $$

$$ \forall p\in P: \mathrm{Si } \sum_{c\in C} Y_{pc} \le 3\to \alpha_{p}=0 $$

In [9]:
# 7 10 y 49 tienen que estar en diferente planta
for planta in PLANTAS:
    model.add_constraint(y[(planta, 7)] + y[(planta, 10)] + y[(planta, 49)] <= 1 , ctname=f"ct6_{planta}")

# La planta 12 tiene que tener al menos a 2 clientes de 1 al 10
model.add_constraint(model.sum(y[(12, cliente)] for cliente in CLIENTES if cliente <= 10) >= 2 , ctname=f"ct7")

# for planta in PLANTAS:
#     model.add_if_then(
#         model.sum(y[(planta, cliente)] for cliente in CLIENTES) >= 4, alpha[planta] == 1
#     )
#     model.add_if_then(
#         model.sum(y[(planta, cliente)] for cliente in CLIENTES) <= 3, alpha[planta] == 0
#     )

docplex.mp.LinearConstraint[ct7](y_12_1+y_12_2+y_12_3+y_12_4+y_12_5+y_12_6+y_12_7+y_12_8+y_12_9+y_12_10,GE,2)

$$ \forall p\in P: \mathcal{M} \times \alpha_{p} \ge \sum_{c\in C} Y_{pc} - 3.5 $$
$$ \forall p\in P: \mathcal{M} \times \alpha_{p} \le \mathcal{M} + \sum_{c\in C} Y_{pc} - 3.5 $$

In [10]:
for planta in PLANTAS:
    model.add_constraint(10 * alpha[planta] >= model.sum(y[(planta, cliente)] for cliente in CLIENTES) - 3.5)
    model.add_constraint(10 * alpha[planta] <= 10 +  model.sum(y[(planta, cliente)] for cliente in CLIENTES) - 3.5)

In [11]:
model.set_time_limit(20)

In [12]:
solution = model.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               20
Tried aggregator 1 time.
MIP Presolve eliminated 151 rows and 74 columns.
MIP Presolve added 17 rows and 17 columns.
MIP Presolve modified 2053 coefficients.
Reduced MIP has 1037 rows, 983 columns, and 6669 nonzeros.
Reduced MIP has 966 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (10.87 ticks)
Probing fixed 1 vars, tightened 1 bounds.
Probing time = 0.02 sec. (3.71 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 48 rows and 1 columns.
Reduced MIP has 989 rows, 982 columns, and 6571 nonzeros.
Reduced MIP has 965 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (5.20 ticks)
Probing time = 0.00 sec. (2.31 ticks)
Clique table members: 888.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: determinis

In [13]:
solution

docplex.mp.solution.SolveSolution(obj=41838,values={x_3:1,x_4:1,x_5:8.30..

In [14]:
model.get_solve_status()

<JobSolveStatus.FEASIBLE_SOLUTION: 1>

In [15]:
model.objective_value

41838.00000000111

In [16]:
for planta in PLANTAS:
    if x[planta].solution_value > 0.99:
        print(f"=== Planta {planta} ====")
        print(f"Capacidad Planta: {CAPACIDAD_PLANTA[planta]}")
        print(f"Cantidad Enviada: {sum(y[(planta, cliente)].solution_value * DEMANDA_CLIENTE[cliente] for cliente in CLIENTES)}")
        print(alpha[planta].solution_value)
        for cliente in CLIENTES:
            if y[(planta, cliente)].solution_value > 0.99:
                print(cliente, end="  ")
        print("\n")

=== Planta 3 ====
Capacidad Planta: 32
Cantidad Enviada: 31.99999999992093
0
7  20  37  

=== Planta 4 ====
Capacidad Planta: 52
Cantidad Enviada: 52.0
0
8  10  39  

=== Planta 6 ====
Capacidad Planta: 95
Cantidad Enviada: 95.0
0
3  47  50  

=== Planta 7 ====
Capacidad Planta: 29
Cantidad Enviada: 28.000000000049585
0
11  17  36  

=== Planta 9 ====
Capacidad Planta: 80
Cantidad Enviada: 80.00000000000003
0
19  46  48  

=== Planta 10 ====
Capacidad Planta: 97
Cantidad Enviada: 96.99999999997252
0
4  14  43  

=== Planta 11 ====
Capacidad Planta: 23
Cantidad Enviada: 23.000000000000004
0
24  26  40  

=== Planta 12 ====
Capacidad Planta: 85
Cantidad Enviada: 85.0
0
1  6  25  

=== Planta 13 ====
Capacidad Planta: 72
Cantidad Enviada: 72.00000000000027
0
21  28  31  

=== Planta 14 ====
Capacidad Planta: 62
Cantidad Enviada: 61.99999999992949
0
12  13  35  

=== Planta 15 ====
Capacidad Planta: 48
Cantidad Enviada: 47.99999999999991
0
5  9  18  

=== Planta 16 ====
Capacidad Planta: 1